In [3]:
import numpy as np
# import copy
import mido
import time
# from importlib import reload
# import fluidsynth
# from IPython.display import Audio, display 

import music21 as m21
import os
import muspy
# import pandas as pd
import sys
# sys.path.insert(0, '/home/prent/Dropbox/Tutorials/coconet-pytorch/coconet-pytorch-csound')

# import subprocess
from numpy.random import default_rng
rng = np.random.default_rng()
# soundfont = '../coconet-pytorch/font.sf2' # you will need to download this from location specified in the github README.md
midi_dir = 'eval'
# CSD_FILE = 'open_samples3.csd'
LOGNAME = 'open_samples3.log'
# WAV_OUT = '/home/prent/Music/sflib/open_samples3.wav'
# CON_OUT = '/home/prent/Music/sflib/open_samples3-t'
model_path = 'eval/TonicNet_epoch-58_loss-0.322_acc-90.745.pt' # this is the one I trained - it's not as good as the one from github
# model_path = 'eval/TonicNet_epoch-58_loss-0.317_acc-90.928.pt' # this is the one from github
downbeat = 1 # all the synthetic chorales out of TonicNet have a downbeat of 1


## Use the TonicNet generator to create and save synthetic chorales as numpy and MIDI files
Steps:
- generate a sythetic chorale as a MIDI file and also a music21 stream.
- convert that to the chorale format, N voices, each with a stream of MIDI note numbers in time_steps like the piano_roll format. 
- evaluate the chorales according to different criteria, including pitch entropy, key, related major and minor keys, length, other factors
- choose some interesting chorales, cherry picked to meet certain criteria, and make some music

In [4]:
# These are pulled from the TonicNet github: 
from eval.sample import sample_TonicNet_random
from eval.utils import plot_loss_acc_curves, indices_to_stream, smooth_rhythm

In [5]:
midi_dir = 'eval/midi_samples'
numpy_dir = 'eval/numpy_chorales'
midi_name = os.path.join(midi_dir,'sample.mid')
keys = ['C','C#','D','E-','E','F','F#','G','A-','A','B-','B']

In [6]:
def print_chorale_metric_report(dirname, start = 0, end = 99_999):
      metrics = []
      print(f'{dirname = }')
      file_count = 0
      report_count = 0
      for filename in os.listdir(dirname):
            if start <= file_count <= end:
                  report_count += 1
                  path = os.path.join(dirname,filename)
                  s = m21.converter.parse(path)
                  fis = s.analyze('key')
                  muspy.read_midi(path, backend='mido', duplicate_note_mode='fifo')
                  music = muspy.read_midi(path) # convert mido to muspy music
                  metric = [os.path.basename(filename), # make a list of metrics
                        muspy.pitch_range(music),
                        muspy.n_pitches_used(music),
                        muspy.n_pitch_classes_used(music), 
                        round(muspy.pitch_entropy(music),3),
                        round(muspy.polyphony(music),3), 
                        round(muspy.polyphony_rate(music),3),
                        round(muspy.scale_consistency(music),3), 
                        round(muspy.pitch_class_entropy(music),3),
                        len(music.tracks),
                        music.get_end_time(), 
                        str(fis)]
                  metrics.append(metric)
            file_count += 1
      print(f'{file_count = }, {report_count = }')
      return metrics

In [7]:
%%timeit
# This takes 26 minutes for 4900 midi files.
are_you_sure = True
if are_you_sure:
      metrics = print_chorale_metric_report('eval/midi_samples', start = 0, end = 5200)

dirname = 'eval/midi_samples'
file_count = 4799, report_count = 4799
dirname = 'eval/midi_samples'
file_count = 4800, report_count = 4800
dirname = 'eval/midi_samples'


In [14]:
print(f'{len(metrics) = }')

len(metrics) = 4799


In [10]:
def sort_key(e):
          return (e[8]) # sort by the value in the slot #8, class entropy,
metrics.sort(key=sort_key, reverse = True) # highest on top
print(f'index\tfile\t\tpitch\tpitch\tpitch\tpitch\tpoly-\tpoly\tscale\tclass\ttracks\tlast\t\tkey')
print(f'num\tname\t\trange\tuse\tclas\tentr\tphony\trate\tconsist\tentropy\tvoice\ttime\t\tsignature')
print(f'array\t0\t\t1\t2\t3\t4\t5\t6\t7\t8\t9\t10\t\t11')
i = 0
list_of_top_chorales = []
for metric in metrics:
    complexity = metric[10] / metric[8]
    if 50_000 < metric[10] < 250_000 and metric[8] > 3.0: # if the last_time of the MIDI file is within this range print the score.
        list_of_top_chorales.append(metric[0])
        print(f'{i}', end='\t')
        print('{:12}'.format(metric[0]),'\t',metric[1],'\t',metric[2],
        '\t',metric[3],'\t',round(metric[4],2),'\t',round(metric[5],2),'\t',metric[6],
        '\t',round(metric[7],2),'\t',round(metric[8],2),'\t',metric[9],'\t',metric[10], ' \t', metric[11])
        i += 1
midi_nums = np.array([int(midi_name[6:midi_name.find('.')]) for midi_name in list_of_top_chorales])
print(f'{midi_nums.shape = }')
np.save('midi_nums.npy', midi_nums)
print(f'{midi_nums[24] = }')
print(f'{list_of_top_chorales[24] = }')
np.save('metrics.npy', metrics)
!ls -lth midi_nums.npy metrics.npy

index	file		pitch	pitch	pitch	pitch	poly-	poly	scale	class	tracks	last		key
num	name		range	use	clas	entr	phony	rate	consist	entropy	voice	time		signature
array	0		1	2	3	4	5	6	7	8	9	10		11
0	sample4036.mid 	 39 	 40 	 12 	 4.98 	 3.91 	 1.0 	 0.63 	 3.56 	 4 	 153600  	 F# major
1	sample4110.mid 	 39 	 38 	 12 	 4.87 	 3.89 	 1.0 	 0.61 	 3.56 	 4 	 60160  	 a minor
2	sample4105.mid 	 36 	 36 	 12 	 4.93 	 3.93 	 1.0 	 0.69 	 3.54 	 4 	 83200  	 f minor
3	sample1285.mid 	 42 	 38 	 12 	 4.88 	 3.86 	 1.0 	 0.66 	 3.53 	 4 	 89088  	 A- major
4	sample4098.mid 	 39 	 38 	 12 	 4.91 	 3.89 	 1.0 	 0.68 	 3.53 	 4 	 84992  	 c# minor
5	sample4239.mid 	 43 	 43 	 12 	 4.96 	 3.92 	 0.997 	 0.7 	 3.52 	 4 	 153600  	 g minor
6	sample4348.mid 	 42 	 38 	 12 	 4.91 	 3.9 	 1.0 	 0.7 	 3.52 	 4 	 85760  	 A- major
7	sample503.mid 	 35 	 34 	 12 	 4.9 	 3.94 	 1.0 	 0.71 	 3.52 	 4 	 96512  	 G major
8	sample1473.mid 	 42 	 40 	 12 	 4.96 	 3.93 	 1.0 	 0.7 	 3.51 	 4 	 124416  	 b minor
9	sampl

In [13]:
print(*[metric for metric in metrics if (metric[11] == "D major")],sep='\n')

['sample4411.mid', 39, 39, 12, 5.005, 3.944, 1.0, 0.706, 3.499, 4, 100608, 'D major']
['sample1844.mid', 37, 35, 12, 4.74, 3.929, 1.0, 0.736, 3.471, 4, 108032, 'D major']
['sample1587.mid', 34, 34, 12, 4.687, 3.912, 1.0, 0.815, 3.353, 4, 111104, 'D major']
['sample4407.mid', 40, 38, 12, 4.709, 3.974, 1.0, 0.828, 3.318, 4, 89088, 'D major']
['sample4333.mid', 40, 37, 12, 4.716, 3.914, 1.0, 0.84, 3.301, 4, 53760, 'D major']
['sample3911.mid', 38, 31, 12, 4.592, 3.97, 1.0, 0.838, 3.296, 4, 84480, 'D major']
['sample3053.mid', 33, 25, 12, 4.401, 3.974, 1.0, 0.865, 3.257, 4, 19456, 'D major']
['sample4022.mid', 41, 30, 12, 4.528, 3.934, 1.0, 0.835, 3.253, 4, 23296, 'D major']
['sample3226.mid', 37, 32, 12, 4.521, 3.924, 1.0, 0.832, 3.239, 4, 36864, 'D major']
['sample1573.mid', 41, 37, 12, 4.65, 3.905, 1.0, 0.877, 3.236, 4, 123904, 'D major']
['sample2970.mid', 41, 37, 12, 4.699, 3.922, 1.0, 0.867, 3.235, 4, 153600, 'D major']
['sample3580.mid', 40, 36, 12, 4.653, 3.959, 1.0, 0.868, 3.225, 